In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
from preprocessing import *

In [2]:
train_file_ds = tf.data.Dataset.list_files(['complete_training_data/*.wav'])
batch_size = 5

train_ds = train_file_ds.map(preprocess_with_resized_mel)#.batch(batch_size)

for x, sampling_rate, label in train_ds.take(1):
    print(x.shape)
    print(f'Data shape: {x.shape} - Sampling rate: {sampling_rate.numpy()} - Label: {label.numpy().decode()}') # se non .batch c'è .decode()

Tensor("Shape:0", shape=(None,), dtype=int32)


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} input image must be of non-zero size
	 [[{{node resize/ResizeBilinear}}]] [Op:IteratorGetNext]